<a href="https://colab.research.google.com/github/seohyun126/Kaggle_script/blob/main/dacon_%EC%9B%94%EA%B0%84_%EC%86%8C%EC%84%A4_%EC%9E%91%EA%B0%80_%EB%B6%84%EB%A5%98_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_NLP_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
from konlpy.tag import Kkma


# NLP 전처리

NLP Preprocessing
- NLP 전처리는 정해진 정답이 업스며 데이터와 목적에 따라 달라짐. 이 과정은 주로 모델의 입력인 단어, 문장, 문서의 vector를 만들기 전에 진행이 됨
- 일반적인 NLP 전처리의 과정을 이번 신문기사 분류 대회에 적용
1. 데이터를 불러온 후 신문기사들을 눈으로 확인하며 특수문자, 불용어, 문장구조에 대한 감을 잡음
2. 문제의 목적과 분석자의 재량에 따라 불용어를 설정하고 리스트에 저장. 이번 대회에서는 특수문자와 조사만 제거해도 어느 정도 높은 정확도를 얻을 수 있음
3. 불용어 이외의 특수 문자를 제거함. 이번 대회에서는 정규표현식 패키지 re를 사용하여 한글과 영어 소문자를 제외한 모든 글자를 제거했음
4. 형태소 분석을 통해 형태소 단위의 토큰으로 분리함. 이때 내가 설정한 불용어들을 결과로 반환해주는 형태소 분석기를 사용해야 함. 예를 들어 조사를 불용어로 설정하였는데 조사를 분리하지 못하는 형태소 분석기는 후보에서 제외해야 함
5. 형태소 단위의 토큰들을 기반으로 리스트에 저장된 불용어를 제거함

## 1. 형태소 분석 (Stemming)
- 형태소 분석이란 단어나 문장의 언어적 속성을 파악하는 것을 의미. 보통 품사의 태깅(PoS)를 통해 이루어지며 한국어 형태소 분석을 위해 konlpy 패키지에 있는 다양한 함수를 이용하여 진행할 수 있음
- 형태소 분석을 하는 이유는 주로 형태소 단위로 의미있는 단어를 가져가고 싶거나 품사 태깅을 통해 형용사나 명사를 추출하고 싶을 때 많이 이용함
- 문장을 띄어쓰기 단위로 분류하여 vectorization하게 되면 같은 의미의 토큰 3개가 서로 다른 vector를 가지게 됨. 이렇게 되면 모델이 세 단어를 각각 다른 단어로 이해함. 하지만 형태소 분석을 통해 '데이콘'이라는 토큰을 추출한다면 앞의 세 단어는 동일한 vector를 갖게 되며 모델이 해당 토큰을 더 잘 학습하는데 도움이 됨
- 형태소 분석은 어쩌면 모델링보다 성능에 더 중요한 영향을 미치는 아주 중요한 과정. 다양한 형태소 분석기를 사용하여 결과 비교하는거 추천

In [ ]:
# 1-1 Kkma()
from konlpy.tag import Kkma
kkma = Kkma()

sentence = '데이콘에서 다양한 컴피티션을 즐기면서 실력있는 데이터 분석가로 성장하세요!!.'

print("형태소 단위로 문장 분리")
print("----------------------")
print(kkma.morphs(sentence))
print(" ")
print("문장에서 명사 추출")
print("----------------------")
print(kkma.nouns(sentence))
print(" ")
print("품사 태킹(PoS)")
print("----------------------")
print(kkma.pos(sentence))

In [ ]:
## 1-2 Okt
from konlpy.tag import Okt
Okt = Okt()

sentence = '데이콘에서 다양한 컴피티션을 즐기면서 실력있는 데이터 분석가로 성장하세요!!.'

print("형태소 단위로 문장 분리")
print("----------------------")
print(Okt.morphs(sentence))
print(" ")
print("문장에서 명사 추출")
print("----------------------")
print(Okt.nouns(sentence))
print(" ")
print("품사 태킹(PoS)")
print("----------------------")
print(Okt.pos(sentence))

** 개인적인 경험에 기반한 결과

- Mecab: 굉장히 속도가 빠르면서도 좋은 분석 결과를 보여줌
- Komoran: 댓글과 같이 정제되지 않은 글에 대해서 먼저 사용해보면 좋음 (오탈자를 어느정도 고려해준다.)
- Kkma: 분석 시간이 오래걸리기 때문에 잘 이용하지 않게 됨
- Okt: 품사 태깅 결과를 Noun, Verb등 알아보기 쉽게 반환해줌
- khaiii: 카카오에서 가장 최근에 공개한 분석기, 성능이 좋다고 알려져 있으며 다양한 실험이 필요

## 2. 표제어 추출 (Lemmatization)

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

sentence = '성장했었다.'

print("품사 태킹(PoS)")
print("----------------------")
print(kkma.pos(sentence))

In [ ]:
sentence = '성장하였었다.'

print("품사 태킹(PoS)")
print("----------------------")
print(kkma.pos(sentence))

## 3. 불용어 제거 (Stopwords Removing)

불용어란 문장에서 큰 의미가 없다고 생각되는 단어. 불용어는 데이터와 문제에 따라 유동적임.

- 예시: "이번에 새롭게 개봉한 영화의 배우들은 모두 훌륭한 연기력과 아름다운 목소리를 갖고 있어!!"

예시 문장에서 감성분석을 진행할 때는 "훌륭한"과 "아름다운"등이 주요 특징으로 사용될 것. 하지만 경우에 따라서는 이러한 형용사들을 제외한 배우들의 연기력과 목소리라는 정보에 집중해야 할 때가 있음. 이럴때는 "훌륭한"과 "아름다운"은 불용어로 정의될 수 있습니다.

In [ ]:
import re
from konlpy.tag import Okt
tokenizer = Okt()
def text_preprocessing(text,tokenizer):

    stopwords = ['을', '를', '이', '가', '은', '는']

    txt = re.sub('[^가-힣a-z]', ' ', text)
    token = tokenizer.morphs(txt)
    token = [t for t in token if t not in stopwords]

    return token

ex_text = "이번에 새롭게 개봉한 영화의 배우들은 모두 훌륭한 연기력과 아름다운 목소리를 갖고 있어!!"
example_pre= text_preprocessing(ex_text,tokenizer)

In [ ]:
# 영어 소문자와 한글을 제외한 모든 문자 제거
# Okt를 이용해 형태소 분석
# 형태소 분석기를 거쳐 나온 결과들 중에 stopwords 리스트에 포함되지 않은 token만 token이라는 리스트에 반환
print(example_pre)

## 4. 대회 적용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/novel_train.csv')

In [ ]:
def text_preprocessing(text_list):

    stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
    tokenizer = Okt() #형태소 분석기
    token_list = []

    for text in text_list:
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) #형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)

    return token_list, tokenizer

#형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 되기 때문입니다.
train['new_article'], okt = text_preprocessing(train['text'])